
## Day 2 – Data Cleaning & Preprocessing


In this stage, we focus on:

Handling missing values

Fixing data types

Removing unnecessary columns

Detecting and handling outliers

Preparing a cleaned dataset for further analysis


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("day1_output.csv")

In [4]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,days_in_hospital,payer_code,...,number_diagnoses,metformin,repaglinide,glimepiride,glyburide,pioglitazone,rosiglitazone,acarbose,insulin,readmitted
0,72091308,20123568,Caucasian,Female,[70-80),1,22,7,7,MC,...,9,No,No,No,No,Up,No,No,Steady,NO
1,72848634,20377854,Caucasian,Female,[60-70),2,1,1,3,MC,...,6,No,No,No,No,No,No,No,Steady,NO
2,73062156,20408121,Caucasian,Female,[90-100),1,1,7,4,MC,...,6,No,No,Steady,No,No,No,No,No,NO
3,73731852,20542797,Caucasian,Male,[70-80),1,2,7,10,MC,...,6,Steady,No,No,No,No,No,No,Steady,NO
4,81355914,7239654,Caucasian,Female,[70-80),1,3,6,12,UN,...,5,No,No,No,No,No,No,No,Steady,NO


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27915 entries, 0 to 27914
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   encounter_id              27915 non-null  int64 
 1   patient_nbr               27915 non-null  int64 
 2   race                      27915 non-null  object
 3   gender                    27915 non-null  object
 4   age                       27915 non-null  object
 5   admission_type_id         27915 non-null  int64 
 6   discharge_disposition_id  27915 non-null  int64 
 7   admission_source_id       27915 non-null  int64 
 8   days_in_hospital          27915 non-null  int64 
 9   payer_code                27915 non-null  object
 10  doctors_dep               27915 non-null  object
 11  num_lab_procedures        27915 non-null  int64 
 12  num_procedures            27915 non-null  int64 
 13  num_medications           27915 non-null  int64 
 14  number_outpatient     

## convert age to numeric 
The age variable was stored as categorical ranges. It was converted to numeric midpoint values to enable statistical analysis and correlation testing.

In [14]:
df[['age_lower','age_upper']] = df['age'].str.extract('(\d+)-(\d+)').astype(int)

df['age_numeric'] = ((df['age_lower'] + df['age_upper']) / 2).astype(int)

df.drop(columns=['age_lower','age_upper'], inplace=True)

In [15]:
df['age_numeric'].dtype

dtype('int64')

In [17]:
df.drop(columns=['age'], inplace=True)

In [20]:
df.rename(columns={'age_numeric': 'age'}, inplace=True)

In [21]:
df['age'].head(10)

0    75
1    65
2    95
3    75
4    75
5    75
6    55
7    35
8    75
9    75
Name: age, dtype: int64

## Standardize Categorical Variables:

In [22]:
df.replace("?", np.nan, inplace=True)

In [25]:
df['payer_code'] = df['payer_code'].fillna("Unknown")

In [28]:
df.duplicated().sum()


np.int64(0)

In [34]:
df = df.rename(columns={'num_medications': 'Total_medicines'})

## handling missing values:


In [30]:
missing_percent = df.isna().sum()
missing_percent.sort_values(ascending=False)

race                        775
additional_diagnosis        387
secondary_diagnosis         125
primary_diagnosis             5
encounter_id                  0
readmitted                    0
insulin                       0
acarbose                      0
rosiglitazone                 0
pioglitazone                  0
glyburide                     0
glimepiride                   0
repaglinide                   0
metformin                     0
number_diagnoses              0
number_inpatient              0
patient_nbr                   0
number_emergency              0
number_outpatient             0
num_medications               0
num_procedures                0
num_lab_procedures            0
doctors_dep                   0
payer_code                    0
days_in_hospital              0
admission_source_id           0
discharge_disposition_id      0
admission_type_id             0
gender                        0
age                           0
dtype: int64

In [31]:
df['race'] = df['race'].fillna("Unknown")
df['additional_diagnosis'] = df['additional_diagnosis'].fillna("Not Recorded")
df['secondary_diagnosis'] = df['secondary_diagnosis'].fillna("Not Recorded")
df = df.dropna(subset=['primary_diagnosis'])

In [32]:
missing_percent = df.isna().mean() * 100
missing_percent.sort_values(ascending=False)

encounter_id                0.0
patient_nbr                 0.0
readmitted                  0.0
insulin                     0.0
acarbose                    0.0
rosiglitazone               0.0
pioglitazone                0.0
glyburide                   0.0
glimepiride                 0.0
repaglinide                 0.0
metformin                   0.0
number_diagnoses            0.0
additional_diagnosis        0.0
secondary_diagnosis         0.0
primary_diagnosis           0.0
number_inpatient            0.0
number_emergency            0.0
number_outpatient           0.0
num_medications             0.0
num_procedures              0.0
num_lab_procedures          0.0
doctors_dep                 0.0
payer_code                  0.0
days_in_hospital            0.0
admission_source_id         0.0
discharge_disposition_id    0.0
admission_type_id           0.0
gender                      0.0
race                        0.0
age                         0.0
dtype: float64

## Perform outlier detection using IQR:

In [44]:
Q1 = df['Total_medicines'].quantile(0.25)
Q3 = df['Total_medicines'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR


outliers = df[(df['Total_medicines'] < lower) | 
              (df['Total_medicines'] > upper)]

print("Number of outliers:", outliers.shape[0])

Number of outliers: 0


In [50]:
Q1 = df['days_in_hospital'].quantile(0.25)
Q3 = df['days_in_hospital'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR


outliers = df[(df['days_in_hospital'] < lower) |
              (df['days_in_hospital'] > upper)]

print("Number of outliers:", outliers.shape[0])

Number of outliers: 439


In [48]:
df['days_in_hospital'].describe()

count    27132.000000
mean         4.178535
std          2.854985
min          1.000000
25%          2.000000
50%          3.000000
75%          6.000000
max         14.000000
Name: days_in_hospital, dtype: float64

## Explanation

Maximum stay = 14 days

13 or 14 days in hospital is normal in real life

Some patients are very sick and need longer treatment

The IQR method showed that hospital stays above 12 days are statistical outliers. However, since longer hospital stays are medically possible and meaningful, these values were kept in the dataset.

In [52]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 27132 entries, 0 to 27914
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   encounter_id              27132 non-null  int64 
 1   patient_nbr               27132 non-null  int64 
 2   race                      27132 non-null  object
 3   gender                    27132 non-null  object
 4   admission_type_id         27132 non-null  int64 
 5   discharge_disposition_id  27132 non-null  int64 
 6   admission_source_id       27132 non-null  int64 
 7   days_in_hospital          27132 non-null  int64 
 8   payer_code                27132 non-null  object
 9   doctors_dep               27132 non-null  object
 10  num_lab_procedures        27132 non-null  int64 
 11  num_procedures            27132 non-null  int64 
 12  Total_medicines           27132 non-null  int64 
 13  number_outpatient         27132 non-null  int64 
 14  number_emergency          2

(27132, 30)

In [53]:
df.isna().sum()

encounter_id                0
patient_nbr                 0
race                        0
gender                      0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
days_in_hospital            0
payer_code                  0
doctors_dep                 0
num_lab_procedures          0
num_procedures              0
Total_medicines             0
number_outpatient           0
number_emergency            0
number_inpatient            0
primary_diagnosis           0
secondary_diagnosis         0
additional_diagnosis        0
number_diagnoses            0
metformin                   0
repaglinide                 0
glimepiride                 0
glyburide                   0
pioglitazone                0
rosiglitazone               0
acarbose                    0
insulin                     0
readmitted                  0
age                         0
dtype: int64

In [54]:
df.shape

(27132, 30)

In [55]:
df.to_csv("day2_output.csv", index=False)

## Day 2 Progress

Fixed data types and converted age ranges to numeric values

Standardized categorical variables

Removed duplicate records

Handled missing values using appropriate strategies

Dropped columns with excessive missing data

Performed outlier detection using IQR method

Applied necessary transformations

Saved cleaned dataset to day2_output.csv